In [26]:
"""Functions for downloading and reading MNIST data."""
import gzip
import os
import urllib
import numpy
import tensorflow as tf

In [27]:
def maybe_download(filename, work_directory):
    """Download the data from Yann's website, unless it's already here."""
    if not os.path.exists(work_directory):
        os.mkdir(work_directory)
    filepath = os.path.join(work_directory, filename)
    if not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
        statinfo = os.stat(filepath)
        print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
    return filepath

def _read32(bytestream):
    dt = numpy.dtype(numpy.uint32).newbyteorder('>')
    return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]

def extract_images(filename):
    """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        magic = _read32(bytestream)
        if magic != 2051:
            raise ValueError('Invalid magic number %d in MNIST image file: %s' \
                             % (magic, filename))
        num_images = _read32(bytestream)
        rows = _read32(bytestream)
        cols = _read32(bytestream)
        buf = bytestream.read(rows * cols * num_images)
        data = numpy.frombuffer(buf, dtype=numpy.uint8)
        data = data.reshape(num_images, rows, cols, 1)
    return data

def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = numpy.arange(num_labels) * num_classes
    labels_one_hot = numpy.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

def extract_labels(filename, one_hot=False):
    """Extract the labels into a 1D uint8 numpy array [index]."""
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        magic = _read32(bytestream)
        if magic != 2049:
            raise ValueError('Invalid magic number %d in MNIST label file: %s' % (magic, filename))
        num_items = _read32(bytestream)
        buf = bytestream.read(num_items)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8)
        if one_hot:
            return dense_to_one_hot(labels)
    return labels

class DataSet(object):
    def __init__(self, images, labels, fake_data=False):
        if fake_data:
            self._num_examples = 10000
        else:
            assert images.shape[0] == labels.shape[0], \
            ("images.shape: %s labels.shape: %s" % (images.shape, labels.shape))
            self._num_examples = images.shape[0]
            # Convert shape from [num examples, rows, columns, depth]
            # to [num examples, rows*columns] (assuming depth == 1)
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])
            # Convert from [0, 255] -> [0.0, 1.0].
            images = images.astype(numpy.float32)
            images = numpy.multiply(images, 1.0 / 255.0)
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
    
    @property
    def images(self):
        return self._images
    
    @property
    def labels(self):
        return self._labels
    
    @property
    def num_examples(self):
        return self._num_examples
    
    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False):
        """Return the next `batch_size` examples from this data set."""
        if fake_data:
            fake_image = [1.0 for _ in xrange(784)]
            fake_label = 0
            return [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = numpy.arange(self._num_examples)
            numpy.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]

def read_data_sets(train_dir, fake_data=False, one_hot=False):
    class DataSets(object):
        pass
    data_sets = DataSets()
    if fake_data:
        data_sets.train = DataSet([], [], fake_data=True)
        data_sets.validation = DataSet([], [], fake_data=True)
        data_sets.test = DataSet([], [], fake_data=True)
        return data_sets
    TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
    TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
    TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
    TEST_LABELS = 't10k-labels-idx1-ubyte.gz'
    VALIDATION_SIZE = 5000
    local_file = maybe_download(TRAIN_IMAGES, train_dir)
    train_images = extract_images(local_file)
    local_file = maybe_download(TRAIN_LABELS, train_dir)
    train_labels = extract_labels(local_file, one_hot=one_hot)
    local_file = maybe_download(TEST_IMAGES, train_dir)
    test_images = extract_images(local_file)
    local_file = maybe_download(TEST_LABELS, train_dir)
    test_labels = extract_labels(local_file, one_hot=one_hot)
    validation_images = train_images[:VALIDATION_SIZE]
    validation_labels = train_labels[:VALIDATION_SIZE]
    train_images = train_images[VALIDATION_SIZE:]
    train_labels = train_labels[VALIDATION_SIZE:]
    data_sets.train = DataSet(train_images, train_labels)
    data_sets.validation = DataSet(validation_images, validation_labels)
    data_sets.test = DataSet(test_images, test_labels)
    return data_sets

In [28]:
# path for logs
logs_path = './tensorflow_logs/example/'
# Import MNIST data
mnist = read_data_sets("./data/", one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [29]:
# Hyperparameters
batch_size = 128
learning_rate = 0.01
num_steps = 200
num_classes = 10
num_input = 784
dropout = 0.75
display_step = 50

In [30]:
# weights
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

# biases
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [31]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# convolution layer
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding="SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# pooling layer
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

# full connection layer
def fullconn(x, W, b):
    x = tf.reshape(x, [-1, W.get_shape().as_list()[0]])
    x = tf.add(tf.matmul(x, W), b)
    return tf.nn.relu(x)

# construct model
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # convolution layer #1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # max pooling layer #1
    conv1 = maxpool2d(conv1, k=2)
    # convolutuon layer #2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # max pooling layer #2
    conv2 = maxpool2d(conv2, k=2)
    # full connection layer #1
    fc1 = fullconn(conv2, weights['wd1'], biases['bd1'])
    # dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    # prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [32]:
# train model
logits = conv_net(X, weights, biases, keep_prob)
# generate prediction
prediction = tf.nn.softmax(logits)
# calculate loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# target
train_op = optimizer.minimize(loss_op)
# evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
saver=tf.train.Saver(max_to_keep=1)

In [33]:
# create a graph
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)

        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, \
                                                                 Y: batch_y, \
                                                                 keep_prob: 1.0})
            print("Step" + str(step) + ", Minibatch Loss=" + \
                  "{:.4f}".format(loss) + " Training Accuracy= " + \
                  "{:.3f}".format(acc))
    
    print("Optimization Finished!")
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images[:256], \
                                                             Y: mnist.test.labels[:256], \
                                                             keep_prob: 1.0}))
    saver.save(sess,'./ckpt/mnist.ckpt')
    print("Model saved!")

Step1, Minibatch Loss=80028.7969 Training Accuracy= 0.203
Step50, Minibatch Loss=420.1198 Training Accuracy= 0.898
Step100, Minibatch Loss=398.3317 Training Accuracy= 0.867
Step150, Minibatch Loss=75.9919 Training Accuracy= 0.930
Step200, Minibatch Loss=229.7367 Training Accuracy= 0.938
Optimization Finished!
Testing Accuracy: 0.9609375
Model saved!


In [36]:
from tensorflow.python import pywrap_tensorflow
with tf.Session() as sess:
    saver = tf.train.Saver() 
    reader = pywrap_tensorflow.NewCheckpointReader('./ckpt/mnist.ckpt')
    var_to_shape_map = reader.get_variable_to_shape_map()
    for key in var_to_shape_map:
        print("tensor_name: ", key)
        print(reader.get_tensor(key)) 

tensor_name:  beta2_power_2
0.8178322
tensor_name:  beta1_power
0.9
tensor_name:  Variable_9
[[[[ 1.26093602e+00  3.24961394e-01  1.80886853e+00 ... -5.01676857e-01
     1.19978058e+00  7.51692653e-01]
   [ 1.10319841e+00  1.11251998e+00  1.16366136e+00 ... -7.26008892e-01
     2.67767161e-01  3.16998690e-01]
   [ 6.29122928e-02 -3.74198139e-01 -1.14522290e+00 ... -1.51905641e-01
    -2.55732059e-01 -3.54132056e-01]
   ...
   [ 1.65651903e-01 -1.04803348e+00 -1.20714104e+00 ... -1.41961062e+00
     1.86899173e+00  9.53436852e-01]
   [ 1.82433054e-01 -7.49769926e-01 -3.65096688e-01 ...  1.83768690e-01
    -1.91769409e+00  2.08866143e+00]
   [-1.73282051e+00  1.66426450e-01  3.54834706e-01 ...  7.13671073e-02
     7.69887984e-01 -1.36207712e+00]]

  [[-6.38617694e-01  4.95748073e-01 -2.85643786e-01 ... -6.73657715e-01
     8.73817503e-01 -1.16518268e-03]
   [ 2.67451954e+00 -2.14140981e-01  1.11593150e-01 ... -3.62360221e-03
    -1.50755435e-01 -9.96900856e-01]
   [ 1.07213271e+00 -9.887